In [5]:
from pyscf import scf,gto,cc
import numpy as np,numpy
from FcMole import FcM

In [6]:
def DeltaV(mol,dL): # dL=[[atmidxs],[atmFcs]]
    mol.set_rinv_orig_(mol.atom_coords()[dL[0][0]])
    dV=mol.intor('int1e_rinv')*dL[1][0]
    for i in range(1,len(dL[0])): 
        mol.set_rinv_orig_(mol.atom_coords()[dL[0][i]])
        dV+=mol.intor('int1e_rinv')*dL[1][i]
    return -dV

In [7]:
#coefficients accurate dL^-8
d1=np.asarray([1/280,-4/105,1/5,-4/5,0,4/5,-1/5,4/105,-1/280])
d2=np.asarray([-1/560,8/315,-1/5,8/5,-205/72,8/5,-1/5,8/315,-1/560])
d3=np.asarray([-7/240,3/10,-169/120,61/30,0,-61/30,169/120,-3/10,7/240])
d4=np.asarray([7/240,-2/5,169/60,-122/15,91/8,-122/15,169/60,-2/5,7/240])
d5=np.asarray([1/6,-3/2,13/3,-29/6,0,29/6,-13/3,3/2,-1/6])
d6=np.asarray([-1/4,3,-13,29,-75/2,29,-13,3,-1/4])
coeffs=np.array([d1,d2,d3,d4,d5,d6])

In [8]:
dl=.05
Ps=[]
Es=[]
Es_hf=[]
for i in range(-4,5):
    fm=FcM(fcs=[i*dl,-i*dl],atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
    mf=scf.RHF(fm)
    mf.scf(dm0=mf.init_guess_by_1e())
    ccalc=cc.CCSD(mf)
    ccalc.run()
    Es_hf.append(mf.e_tot-mf.energy_nuc())
    Es.append(ccalc.e_tot-mf.energy_nuc())
    Ps.append(np.einsum('pi,ij,qj->pq', mf.mo_coeff, ccalc.make_rdm1(), mf.mo_coeff))

converged SCF energy = -112.742145890619
E(CCSD) = -112.849666107281  E_corr = -0.107520216661614
converged SCF energy = -112.351169769144
E(CCSD) = -112.4638002865437  E_corr = -0.1126305173996812
converged SCF energy = -111.967431228079
E(CCSD) = -112.0851825848047  E_corr = -0.1177513567254177
converged SCF energy = -111.5909932961
E(CCSD) = -111.7138212760532  E_corr = -0.1228279799533206
converged SCF energy = -111.221914650137
E(CCSD) = -111.3497197533591  E_corr = -0.1278051032223013
converged SCF energy = -110.860249698717
E(CCSD) = -110.9928772044254  E_corr = -0.1326275057079391
converged SCF energy = -110.506048710199
E(CCSD) = -110.6432890073989  E_corr = -0.1372402971997884
converged SCF energy = -110.15935797469
E(CCSD) = -110.3009477592215  E_corr = -0.1415897845311751
converged SCF energy = -109.820219990832
E(CCSD) = -109.9658441981601  E_corr = -0.1456242073277113


In [9]:
mol0=gto.M(atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
dV=DeltaV(mol0,[[0,1],[1,-1]])
mf0=scf.RHF(mol0)
mf0.scf()
cc0=cc.CCSD(mf0)
cc0.run()
E0=cc0.e_tot
P0_hf=mf0.make_rdm1()
P0=np.einsum('pi,ij,qj->pq', mf0.mo_coeff, cc0.make_rdm1(), mf0.mo_coeff.conj())

converged SCF energy = -111.221914650137
E(CCSD) = -111.3497197447145  E_corr = -0.12780509457712


In [10]:
np.einsum('ij,ij',P0,dV)

6.235764203543833

In [11]:
np.sum(d1*Es)/dl

6.257050244681254

In [12]:
(Es[5]-Es[3])/2/dl

6.257059763896393

In [13]:
np.einsum('ij,ij',P0_hf,dV)

6.355230016182917

In [14]:
np.sum(d1*Es_hf)/dl

6.355229384767897

In [15]:
molT=FcM(fcs=[1,-1],atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
mfT=scf.RHF(molT)
mfT.scf(dm0=mf.init_guess_by_1e())
ccT=cc.CCSD(mfT)
ccT.run()
ET=ccT.e_tot
PT=np.einsum('pi,ij,qj->pq', mfT.mo_coeff, ccT.make_rdm1(), mfT.mo_coeff)

converged SCF energy = -105.444757226602
E(CCSD) = -105.5894764419116  E_corr = -0.1447192153099631


In [16]:
np.einsum('ij,ij',P0,dV),np.einsum('ij,ij',mf0.make_rdm1(),dV)

(6.235764203543833, 6.355230016182917)

In [17]:
#dE/dZ^(n+1) = \int dP/dZ^n * dV
e=E0+np.einsum('ij,ij',P0,dV) +(mfT.energy_nuc()-mf0.energy_nuc())
print(e,np.einsum('ij,ij',P0,dV) )
for i in range (6):
    dP=np.einsum('xij,x->ij',Ps,coeffs[i]/(dl)**(i+1))
    e+=np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2)
    print(e,np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2))

-104.63776506498017 6.235764203543833
-105.60824526942235 -0.9704802044421782
-105.58461920742558 0.0236260619967833
-105.56345184232883 0.021167365096740914
-105.59160025245468 -0.028148410125853045
-105.59377074557482 -0.0021704931201386372
-105.59846271609632 -0.0046919705215124384


In [18]:
#expanding energies
e=E0+(mfT.energy_nuc()-mf0.energy_nuc())
for i in range (6):
    e+=np.sum(coeffs[i]*Es)/np.math.factorial(i+1)/dl**(i+1)
    print(e,np.sum(coeffs[i]*Es)/np.math.factorial(i+1)/dl**(i+1))

-104.61647902384276 6.257050244681254
-105.59214878750562 -0.9756697636628673
-105.5882954237981 0.0038533637075251655
-105.56187460065398 0.0264208231441169
-105.58009045223666 -0.01821585158268135
-105.67299728638362 -0.09290683414696506


In [19]:
import pyscf
from pyscf import grad


In [20]:
from pyscf.cc import ccsd_grad


ImportError: cannot import name 'ccsd_grad' from 'pyscf.cc' (/home/giorgiod/miniconda3/lib/python3.7/site-packages/pyscf/cc/__init__.py)

In [ ]:
pyscf.__version__
